# evaluate_basis() is wrong for "hermite" shape functions

That is: for those coming from Derivative evaluations. In $[0,1]$ with 10 subdivisions they are off by a factor of exactly 10, i.e. the Jacobian of the transformation. This seems to imply that `*values = (J[0]*h_tmp0);` is wrong in the generated code.

Also: in the same setting, derivatives of the lagrange basis functions seem to lack a factor 0.1, i.e. multiplication by the Jacobian. But these should require no product!?

** Am I already doing the transformation in FIAT?? Is this even possible? ** (How would it know what the Jacobian is?)

To see this: compate the output of [Hermite shape functions in 1D](#Hermite-shape-functions-in-1D) below with the output of [Plotting shape functions](#Plotting-shape-functions). This probably means that the computation in `evaluatebasis.py - _compute_values()` is wrong!

# Current

* Implement (nodal) interpolation for Hermite: how should one compute the derivatives of the function to be interpolated?
* Fix the hack (in particular remove the check for `isinstance(elem, CubeHermite)` in `representation.py` and generalise to 3D, etc.
* Dec 2016: I don't understand why there is no problem with the Argyris element any more since I pulled the latest changes from branch `next`. Before I had the same issues as with the CubicHermite...
* <strike>Implement the trafo for the derivatives.</strike>
* <strike>Fix the issue with the mappings in the quadratures.</strike>
* <strike>Fix `_tabulate_dofs()` in `representation.py` to add the coordinates where dofs associated to derivative evaluations are .</strike>

# Some nomenclature

**Finite element** $(K,P,N)$: $K$ is the geometry, $P$ some polynomial space over $K$ and $N \subset P'$ a set of functions in the dual space of $P$ (the _degrees of freedom_) such that its dual $B \subset P$, i.e. a set of polynomials $p_i \in P$ such that $L_i(p_j) = \delta_{i j}$ for all $L_i \in N$, is a basis for all of $P$.

**dof** in ffc: element of the dual space $N=P'$. From the docs: 
>Each dof $L$ [in an element] is assumed to act on a field $f$ in the following manner: $$L(f) = w_{j, k} f_k(x_j),$$ where $w$ is a set of weights, $j$ is an index set corresponding to the number of points involved in the evaluation of the functional, and $k$ is a multi-index set with rank corresponding to the value rank of the function $f$. For common degrees of freedom such as point evaluations and directional component evaluations, there is just one point. However, for various integral moments, the integrals are evaluated using quadrature. The number of points therefore correspond[s] to the quadrature points. The points $x_j$, weights $w_{j, k}$ and components $k$ are extracted from FIAT (`functional.pt_dict`) in the intermediate representation stage.

By extension, **dof** is also used for the coefficients of the expansion of any function $f \in V_h$, where $V_h$ is the finite dimensional space built from all finite elements.

**Intermediate representation**: A dictionary built in `ffc/representation.py` for later code generation.

# Related commits and PRs

* [This commit](https://bitbucket.org/fenics-project/ffc/commits/8c96e6646aa7c42365a954e79d5df7ec4066b1e2) implements the "double covariant" transform in ffc. Modifies:
  * `ffc/evaluatebasis.py`
  * `ffc/evaluatebasisderivatives.py`
  * `ffc/evaluatedof.py`
  * `ffc/interpolatevertexvalues.py`
  * `ffc/quadrature/quadraturetransformer.py`
  * `uflacs/backends/ufc/evaluatebasis.py`

* [This PR](https://bitbucket.org/fenics-project/ufl/pull-requests/55/support-for-hellan-herrmann-johnson/diff) adds support for an element in UFL using the double covariant trafo. Modifies:
  * `ufl/algorithms/apply_function_pullbacks.py`
  * `ufl/finiteelement/elementlist.py`

# Intro

See hermite.tm...

# Hermite shape functions in 1D

We choose a monomial basis $G=\{1, x, x^2, x^3\}$ to express the shape functions in. The degrees of freedom $L \subset P_3(T)'$ are point evaluation and differentiation at $0$ and $1$. We build the Vandermonde matrix $V_{i j} = L_i(G_j)$ and invert it. The columns of $V^{-1}$ are the coefficients of the shape functions expressed in the basis $G$.

In [ ]:
import numpy as np
import matplotlib.pyplot as pl
%matplotlib inline

sh = [lambda x: 1     - 3*x*x + 2*x*x*x,
      lambda x:     x - 2*x*x +   x*x*x,
      lambda x:         3*x*x - 2*x*x*x,
      lambda x:       -   x*x +   x*x*x]

shd = [lambda x: - 6*x + 6*x*x,
       lambda x:     1 - 4*x +   3*x*x,
       lambda x:         6*x - 6*x*x,
       lambda x:       -   2*x +   3*x*x]

xx = np.linspace(0, 1, 100)
for i, f in enumerate(sh):
    pl.plot(xx, f(xx), label='sh%d' % i)
_ = pl.legend()
pl.figure()
for i, f in enumerate(shd):
    pl.plot(xx, f(xx), label='sh%d\'' % i)
_ = pl.legend()

# Hermite shape functions in 2D

We repeat the procedure for 1D, but instead of manually computing $V$, we use Automatic Differentiation and a bit of Python to do the job. Now $G=\{1, x, y, x^2, x y, y^2, x^3, x^2 y, x y^2, y^3\}$.

In [ ]:
import autograd as ad
import autograd.numpy as np

# Partial derivatives
def dx(f):
    return ad.grad(f, 0)
def dy(f):
    return ad.grad(f, 1)

# Monomial basis for $P_3(R^2)$
G = [lambda x,y: 1, lambda x,y: x, lambda x,y: y, lambda x,y: x**2, lambda x,y: x*y, lambda x,y: y**2,
     lambda x,y: x**3, lambda x,y: x**2*y, lambda x,y: x*y**2, lambda x,y: y**3]

# String representation of monomial basis for the code output
S = ["1", "x[0]", "x[1]", "x[0]**2", "x[0]*x[1]", "x[1]**2",
     "x[0]**3", "x[0]**2*x[1]", "x[0]*x[1]**2", "x[1]**3"]

# Hermite degrees of freedom on the reference triangle [(0,0),(1,0),(0,1)]:
# point evaluation and partial derivatives at each vertex, plus evaluation at the barycenter.
L = [lambda f: f(0., 0.), lambda f: dx(f)(0., 0.), lambda f: dy(f)(0., 0.),
     lambda f: f(1., 0.), lambda f: dx(f)(1., 0.), lambda f: dy(f)(1., 0.),
     lambda f: f(0., 1.), lambda f: dx(f)(0., 1.), lambda f: dy(f)(0., 1.),
     lambda f: f(1./3., 1./3.)]

def create_shape_functions_code(basis, basis_strings, dofs):
    """Create python code with the shape functions.
    This can be used for plotting or be pasted into ffc/tests/unit/test_elements.py."""
    
    n = len(basis)
    assert n == len(dofs) == len(basis_strings) , "Dimensions don't match."

    # Build Vandermonde matrix
    V = np.zeros((n, n))
    for i, j in np.ndindex(V.shape):
        V[i,j] = dofs[i](basis[j])

    # Invert and read linear combinations of basis functions from G from the columns
    Vinv = np.linalg.inv(V)

    s = "sh = ["
    for j in range(n):
        s += "lambda x: "
        l = []
        for i in range(n):
            coeff = Vinv[i,j]
            if not np.isclose(coeff, 0):
                sign = "+" if coeff > 0 else "-"
                coeff = np.round(np.abs(coeff),0)
                prefix = "%s %d *" % (sign, coeff) if not np.isclose(coeff, 1) else sign
                l.append("%s %s" % (prefix, basis_strings[i]))
        s += " ".join(l) + ",\n      "
    s = s[:-8]    # Remove last ",\n      "
    return s + "]"

print(create_shape_functions_code(G, S, L))

We can just copy & paste the output above to plot the shape functions:

In [ ]:
sh = [lambda x: + 1 - 3 * x[0]**2 - 13 * x[0]*x[1] - 3 * x[1]**2 + 2 * x[0]**3 + 13 * x[0]**2*x[1] + 13 * x[0]*x[1]**2 + 2 * x[1]**3,
      lambda x: + x[0] - 2 * x[0]**2 - 3 * x[0]*x[1] + x[0]**3 + 3 * x[0]**2*x[1] + 2 * x[0]*x[1]**2,
      lambda x: + x[1] - 3 * x[0]*x[1] - 2 * x[1]**2 + 2 * x[0]**2*x[1] + 3 * x[0]*x[1]**2 + x[1]**3,
      lambda x: + 3 * x[0]**2 - 7 * x[0]*x[1] - 2 * x[0]**3 + 7 * x[0]**2*x[1] + 7 * x[0]*x[1]**2,
      lambda x: - x[0]**2 + 2 * x[0]*x[1] + x[0]**3 - 2 * x[0]**2*x[1] - 2 * x[0]*x[1]**2,
      lambda x: - x[0]*x[1] + 2 * x[0]**2*x[1] + x[0]*x[1]**2,
      lambda x: - 7 * x[0]*x[1] + 3 * x[1]**2 + 7 * x[0]**2*x[1] + 7 * x[0]*x[1]**2 - 2 * x[1]**3,
      lambda x: - x[0]*x[1] + x[0]**2*x[1] + 2 * x[0]*x[1]**2,
      lambda x: + 2 * x[0]*x[1] - x[1]**2 - 2 * x[0]**2*x[1] - 2 * x[0]*x[1]**2 + x[1]**3,
      lambda x: + 27 * x[0]*x[1] - 27 * x[0]**2*x[1] - 27 * x[0]*x[1]**2]
import matplotlib.pyplot as pl
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
%matplotlib inline

npoints = 100
X = np.linspace(0, 1, npoints)
Y = np.linspace(0, 1, npoints)
xx, yy = np.meshgrid(X,Y)
for i in range(10):
    fig = pl.figure() #figsize=(12,10))
    #ax = fig.add_subplot(3, 3, i+1, projection='3d', label="th%d" % i)
    ax = fig.gca(projection='3d', label="sh%d" % i)
    zz = np.zeros_like(xx)
    for m in range(npoints):
        for n in range(npoints):
            zz[m,n] = sh[i]([xx[m,n], yy[m,n]]) if n <= npoints - m else np.nan
    vmin, vmax = np.min(zz[~np.isnan(zz)]), np.max(zz[~np.isnan(zz)])
    ax.plot_wireframe(xx, yy, zz, color='grey', linewidth=0.2, rstride=2, cstride=2)
    #ax.plot_surface(xx, yy, zz, cmap=cm.bone, linewidth=0, rstride=5, cstride=5, vmin=-0.5, vmax=1.0)
    # Get rid of the panes
    ax.w_xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.w_yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.w_zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    
    ax.xaxis.set_ticks([0.,1.])
    ax.yaxis.set_ticks([0.,1.])
    ax.zaxis.set_ticks([vmin, 0, vmax])
    pl.savefig("hermite2d-w-%d.eps" % i)

# Hermite shape functions in 3D

Now $G=\{1, x, y, z, x^2, y^2, z^2, x y, x z, y z, x^2 y, x^2 z, y^2 z, x y^2, x z^2, y z^2, x y z, x^3, y^3, z^3\}$, i.e. $\text{dim}\ P_3(\mathbb{R}^3) = 20$.

In [ ]:
import autograd as ad
import autograd.numpy as np

# Partial derivatives
def dx(f):
    return ad.grad(f, 0)
def dy(f):
    return ad.grad(f, 1)
def dz(f):
    return ad.grad(f, 2)

# Monomial basis for $P_3(R^3)$
G = [lambda x,y,z: 1,
     lambda x,y,z: x,      lambda x,y,z: y,      lambda x,y,z: z,
     lambda x,y,z: x**2,   lambda x,y,z: y**2,   lambda x,y,z: z**2,
     lambda x,y,z: x*y,    lambda x,y,z: y*z,    lambda x,y,z: x*z,
     lambda x,y,z: x**2*y, lambda x,y,z: x**2*z, lambda x,y,z: y**2*z,
     lambda x,y,z: x*y**2, lambda x,y,z: x*z**2, lambda x,y,z: y*z**2,
     lambda x,y,z: x*y*z,
     lambda x,y,z: x**3,   lambda x,y,z: y**3,   lambda x,y,z: z**3]

# String representation of monomial basis for the code output
S = ["1",
     "x[0]", "x[1]", "x[2]" ,
     "x[0]**2", "x[1]**2", "x[2]**2",
     "x[0]*x[1]", "x[1]*x[2]", "x[0]*x[2]",
     "x[0]**2*x[1]", "x[0]**2*x[2]", "x[1]**2*x[2]",
     "x[0]*x[1]**2", "x[0]*x[2]**2", "x[1]*x[2]**2",
     "x[0]*x[1]*x[2]",
     "x[0]**3", "x[1]**3", "x[2]**3"]

# Hermite degrees of freedom on the reference tetrahedron [(0, 0, 0), (1, 0, 0), (0, 1, 0), (0, 0, 1)]:
# point evaluation and partial derivatives at each vertex, plus evaluation at the barycenter of each face.
# NOTE that we need to use the same ordering as in the FIAT code in order for the tests to pass.
# To see in which sequence the barycenters were created use:
#import FIAT
#e = FIAT.CubicHermite(FIAT.reference_element.UFCTetrahedron(), 3)
#ll = e.dual_basis()
#for l in ll:
#    print(l.pt_dict.keys())

L = [lambda f: f(0., 0., 0.), lambda f: dx(f)(0., 0., 0.), lambda f: dy(f)(0., 0., 0.), lambda f: dz(f)(0., 0., 0.),
     lambda f: f(1., 0., 0.), lambda f: dx(f)(1., 0., 0.), lambda f: dy(f)(1., 0., 0.), lambda f: dz(f)(1., 0., 0.),
     lambda f: f(0., 1., 0.), lambda f: dx(f)(0., 1., 0.), lambda f: dy(f)(0., 1., 0.), lambda f: dz(f)(0., 1., 0.),
     lambda f: f(0., 0., 1.), lambda f: dx(f)(0., 0., 1.), lambda f: dy(f)(0., 0., 1.), lambda f: dz(f)(0., 0., 1.),
     lambda f: f(1./3., 1./3., 1./3.),
     lambda f: f(0., 1./3., 1./3.), lambda f: f(1./3., 0., 1./3.), lambda f: f(1./3., 1./3., 0.)]

print(create_shape_functions_code(G, S, L))

For completeness, here's the code for the one dimensional case:

In [ ]:
# Monomial basis for $P_3(R)$
G = [lambda x: 1, lambda x: x, lambda x: x**2, lambda x: x**3]

# String representation of monomial basis
S = ["1", "x[0]", "x[0]**2", "x[0]**3"]

# Hermite degrees of freedom on the reference interval [(0,0),(1,0)]:
# Point evaluation and partial derivatives at each endpoint
L = [lambda f: f(0.), lambda f: dx(f)(0.), lambda f: f(1.), lambda f: dx(f)(1.)]

print(create_shape_functions_code(G, S, L))

# Setup

In [ ]:
from __future__ import print_function
from dolfin import *
import ufl
import ffc
import utils
import numpy as np
import matplotlib.pyplot as pl
%matplotlib inline

# Force quadrature representation (we only implement this for now)
parameters["form_compiler"]["representation"] = "quadrature"

# Generate code for evaluate_basis_derivatives for testing
parameters["form_compiler"]["no-evaluate_basis_derivatives"] = False

TEST_DIMENSION = 1

In [ ]:
ffc.log.set_level(ffc.log.DEBUG)
#ffc.log.add_logfile("/tmp/fenics.log")
#ffc.__version__, ffc.supported_elements

In [ ]:
if TEST_DIMENSION == 1:
    mesh = UnitIntervalMesh(10)
elif TEST_DIMENSION == 2:
    mesh = UnitSquareMesh(10, 10)
elif TEST_DIMENSION == 3:
    mesh = UnitCubeMesh(10, 10, 10)

V = FunctionSpace(mesh, "Hermite", 3)

After a quick hack we were able to create the Hermite UFL element with

```python
elem = ufl.FiniteElement('Hermite', mesh.ufl_cell(), 3)
```

We did this first to find out what happens at the FFC level when we try to build a `FunctionSpace`. 

**FIXME** We need to specify the degree even though only CubicHermite elements are supported as of yet.

Creating a `FunctionSpace` with Hermite elements involves (at least):

1. We must "register" the Hermite element in `ufl/finitelements/elementlist.py` in order to be able to use the string argument "Hermite".

2. For each basis element of the element which is a derivative (i.e. two at every vertex of the triangle in 2D) we collect in the *intermediate representation* (`ir`) the coefficients of *all* basis elements which are derivatives (i.e. two at every vertex of the triangle in 2D). This is done in `ffc/representation.py`, `_evaluatebasis()` by adding the coefficients to `dof_data`.

3. Inside `ffc/evaluatebasis.py` we use these coefficients and the Jacobian to apply the transformation in `_compute_values()`, inside a new case for mapping "hermite". **FIXME:** Maybe I should change the Hermite mapping to act on point evaluations (as the identity) and directional derivatives (as a linear combination using the domain transformation) in order to have a more "homogeneous" implementation (i.e. without as many cases, etc.). That is: reuse the idea of the matrix $H$ as in the quadratures.

4. Interpolation cannot work without computing derivatives of arbitrary `ufc::function`s, see below.

5. Quadratures also must feature the Hermite transform. A naive approach yields unwieldy expressions with hundreds of terms for even the simplest of forms.

6. ...

# First tests (finish)

A minor sanity check for the degrees:

In [ ]:
import FIAT

ufl_element = ufl.FiniteElement('Hermite', mesh.ufl_cell(), 3)
fiat_element = ffc.fiatinterface._create_fiat_element(ufl_element)

print(ufl_element.degree(), fiat_element.degree(), isinstance(fiat_element, FIAT.hermite.CubicHermite))

Now check the types of mappings for each dof in the element:

In [ ]:
print(fiat_element.mapping())

**NOTE:** Maybe I should use the same idea as I did in the quadratures of collecting both affine and hermite trafos into one $(d+1) \mathrm{x} (d+1)$ matrix

# FFC  element compilation

In [ ]:
import ffc
def compile_element(mesh, element):
    family = element.family()
    dim = mesh.ufl_domain().topological_dimension()
    with open("/tmp/%s-%dD.h" % (family, dim), "wt") as f:
        out = ffc.compile_element(element, prefix=family[:3].lower())
        f.write(out[0])

In [ ]:
her = FiniteElement("Hermite", cell=V.mesh().ufl_cell())
compile_element(UnitIntervalMesh(1), her)
compile_element(UnitSquareMesh(1,1), her)
compile_element(UnitCubeMesh(1,1,1), her)

## Mixed elements

In [ ]:
compile_element(UnitIntervalMesh(2),
                MixedElement([FiniteElement("Lagrange", "interval", 2),
                              FiniteElement("Hermite", "interval", 3)]))

# Plotting shape functions

In [ ]:
mesh = UnitTriangleMesh()
mesh = refine(refine(refine(mesh)))
cell_id = mesh.bounding_box_tree().compute_first_entity_collision(Point(np.array([0.1, 0.1])))
cell = Cell(mesh, cell_id)

# The following two coincide. When do they not?
print("Cell.get_coordinate_dofs() returns the physical coordinates associated to the DOFs in the cell.")
print(cell.get_coordinate_dofs())
print("Cell.get_vertex_coordinates() returns the physical coordinates of the vertices in the cell.")
print(cell.get_vertex_coordinates())

In [ ]:
e = V.element()
val = np.zeros(1)
points = np.arange(0, 1, 0.001)
values = np.zeros((e.space_dimension(), points.size))
derivs = np.zeros((e.space_dimension(), points.size))
for i in range(e.space_dimension()):
    for j, x in enumerate(points):
        x = np.array([x])
        pt = Point(x)
        cell_id = V.mesh().bounding_box_tree().compute_first_entity_collision(pt)
        cell = Cell(V.mesh(), cell_id)
        coordinate_dofs = cell.get_coordinate_dofs()
        e.evaluate_basis(i, val, x, coordinate_dofs, cell.orientation())
        values[i,j] = val
        e.evaluate_basis_derivatives(i, 1, val, x, coordinate_dofs, cell.orientation())
        derivs[i,j] = val

pl.figure()#figsize=(8,6))
for i in range(4):
    pl.plot(points[:101], values[i,:101], label="th%d" % i)
_ = pl.legend()

pl.figure()#figsize=(8,6))
for i in range(4):
    pl.plot(points[:101], derivs[i,:101], label="th%d'" % i)
_ = pl.legend()

# FIXME: The following computations should be made over the same grid!
#See: 
#pl.scatter(10*points[:10], np.zeros(10))
#pl.scatter(xx[:10], np.zeros(10)+0.005)


#for i in range(4):
#    if not np.allclose(values[i,:101], sh[i](10*points[:101])):
#        print("Shape functions %d differ" % i)

#for i in range(4):
#    if not np.allclose(derivs[i,:101], shd[i](10*points[:101])):
#        print("Derivatives of shape functions %d differ by %f" % 
#              (i, np.linalg.norm(derivs[i,:101] - shd[i](10*points[:101]), np.inf)))

# Basis evaluation at points

Here we test that the shape functions fulfill the "delta property"

$$ \partial_j \phi_{\alpha_i} ( v_{\beta} ) = \delta_{\alpha \beta} \delta_{i j}, $$

where $i, j \in \{0,1,...,d\}, \partial_0 \ f = f$, and $\alpha, \beta$ run over the set of indices of vertices of the simplex in $\mathbb{R}^d$.

In [ ]:
def evaluate_basis_at_points(V, points):
    """ 
    Arguments:
        V: FunctionSpace
        xx: np.array of points"""

    def highlight(s, num):
        trafo = utils.yellow if np.isclose(num, 1.) else lambda x:x
        print(trafo(s + "%.2f" % np.round(num, 2)))
        
    element = V.element()
    mesh = V.mesh()
    # Array to store the values
    valdim = element.value_dimension(0)
    assert valdim == 1, "Real valued shape functions expected."
    tdim = element.topological_dimension()
    values = np.zeros(valdim, dtype=np.double)
    derivs = np.zeros(valdim * tdim, dtype=np.double)
    print("DOF, [derivative] @ [x, y] = value   (only if any of DOF or derivatives != 0)\n")
    for xyz in points:
        xyz = np.array(xyz, dtype=np.double)
        pt = Point(xyz) 
        cell_id = mesh.bounding_box_tree().compute_first_entity_collision(pt)
        cell = Cell(mesh, cell_id)
        # FIXME: why get_vertex_coordinates() or get_coordinate_dofs() ?
        #coordinate_dofs = cell.get_vertex_coordinates()
        coordinate_dofs = cell.get_coordinate_dofs()

        print("***************************")
        for i in range(element.space_dimension()): 
            element.evaluate_basis(i, values, xyz, coordinate_dofs, cell.orientation())
            element.evaluate_basis_derivatives(i, 1, derivs, xyz, coordinate_dofs, cell.orientation())
            if not np.isclose(values, 0.) or not np.allclose(derivs, 0.):
                highlight("%d   @ %s = " % (i, xyz), values)
                #else:
                for j in range(tdim):
                    highlight(" ,%d @ %s = " % (j, xyz), derivs[j])

In [ ]:
evaluate_basis_at_points(V, V.mesh().coordinates())

In [ ]:
plot(V.mesh())

# Playing with dofs

We print some information on dofs (**todo** explain this)

In [ ]:
from dolfin import *
import numpy as np
def do_stuff(V):
    """ V: FunctionSpace """
    mesh = V.mesh()
    element = V.element()
    dofmap = V.dofmap()
    for cell in cells(mesh):
        print("At cell: %s" % cell)
        print(np.round(element.tabulate_dof_coordinates(cell), 3))
        print("\tCell DOFs: %s" % dofmap.cell_dofs(cell.index()))
        print("\tCoordinate DOFs: %s" % cell.get_coordinate_dofs())

In [ ]:
do_stuff(V)

# Projection

Simple projection of a constant function already shows some weird stuff. I think `project()` is using the right norm (it should fetch it from FIAT's declaration that HER3 is in $H^1$), but maybe there's more to it.

In [ ]:
f = Constant(2.0)
u = project(f, V)
_ = plot(u, cmap='bone')
print("f is a %s and u is a %s" % (f.ufl_element(), u.ufl_element()))

Note how the projection oscillates when we are not at grid vertices. However total error seems to be within reasonable bounds for double precision ($10^{-13}$)

In [ ]:
if V.element().topological_dimension() == 2:  # This test only works in 2D
    xy = V.mesh().coordinates()
    grid_xx = xy[:11,0]
    xx = np.arange(0, 1, 0.01)
    y = 0.97
    pl.plot(xx, [u(x, y) for x in xx])
    pl.scatter(grid_xx, [u(x, y) for x in grid_xx], c='r')
    _ = pl.xlim(-0.01,1.01)
    
    vals = u.compute_vertex_values(V.mesh())[:, None]
    print("Total error: %.3g" % np.linalg.norm(2. - vals))

Here are a couple more examples. Notice that we first declare the function to be projected onto $V$ as living in another finite element space.

In [ ]:
tdim = V.element().topological_dimension()
if tdim == 1:
    f = Expression("sqrt(x[0])*x[0]", degree=3)
    u = project(f, V)
    vals = u.compute_vertex_values(V.mesh())[:, None]
    coordinates = V.mesh().coordinates()
    errors = vals - np.sqrt(coordinates)*coordinates
    _ = pl.plot(coordinates, errors)
elif tdim == 2:
    f = Expression("x[0]*x[1]", degree=3)
    u = project(f, V)
    vals = u.compute_vertex_values(V.mesh())
    coordinates = V.mesh().coordinates()
    errors = vals - coordinates.T[0] * coordinates.T[1]
    plot(u, cmap='bone')
    pl.scatter(coordinates.T[0], coordinates.T[1], c=errors, s=20, cmap='hot', linewidths=0)
    pl.axes().set_aspect('equal')
print("f is a %s and u is a %s" % (f.ufl_element(), u.ufl_element()))
print("Total error: %.3g" % np.linalg.norm(errors))

In [ ]:
_ = pl.plot(errors)

Here's another example, with a derivative. Notice that we need to explicitly declare the domain for $f$ or Expression won't be able to infer its dimension.

In [ ]:
f = Expression("sqrt((x[0]-0.5)*(x[0]-0.5)+(x[1]-0.5)*(x[1]-0.5))", degree=3, domain=u.ufl_domain())
df = f.dx()
u = project(df, V)
_ = plot(u, cmap='bone')
print("df is a %s and u is a %s" % (df.ufl_element(), u.ufl_element()))

A final example with Hermite shape functions (**FIXME** are these even right?)

In [ ]:
# These three are for the triangle {(-1,-1), (1,-1), (-1,1)}
phi1 = Expression("(7/8)*x[0] + (7/8)*x[1] + (13/8)*x[0]*x[0] + (13/4)*x[0]*x[1]"
                  "+ (13/8)*x[1]*x[1] + (1/4)*x[0]*x[0]*x[0] + (13/8)*x[0]*x[0]*x[1]"
                  "+ (13/8)*x[0]*x[1]*x[1] + (1/4)*x[1]*x[1]*x[1]", degree=3)
phi2 = Expression("1/2 + (13/8)*x[0] + (7/8)*x[1] + (7/8)*x[0]*x[0] + (7/4)*x[0]*x[1]"
                  "+ (7/8)*x[1]*x[1] - (1/4)*x[0]*x[0]*x[0] + (7/8)*x[0]*x[0]*x[1]"
                  "+ (7/8)*x[0]*x[1]*x[1]", degree=3)
phi3 = Expression("1/2 + (7/8)*x[0] + (13/8)*x[1] + (7/8)*x[0]*x[0] + (7/4)*x[0]*x[1]"
                  "+ (7/8)*x[1]*x[1] + (7/8)*x[0]*x[0]*x[1] + (7/8)*x[0]*x[1]*x[1]"
                  "- (1/4)*x[1]*x[1]*x[1]",
                 degree=3)

#mesh = refine(refine(refine(refine(UnitTriangleMesh()))))
#elem = ufl.FiniteElement('Hermite', mesh.ufl_cell(), 3)
#W = FunctionSpace(mesh, elem)

v = project(phi1, V)
_ = plot(v, cmap='bone')

## Projecting from the same space

Up until now, we've declared $f$ to live in a different space, then projected. If we take $f \in V$, then a call to `project()` most likely tries to call `interpolate()` instead. This calls `create_function()` at some point, which requires handling of different transformations. After brute-hacking that to accept Hermite trafos, we hit the problem that `evaluate_dof()` (i.e. interpolation) cannot be implemented for Hermite elements because we are not handling the derivatives (yet), so I disabled it by throwing an exception:

In [ ]:
f = Expression("x[0]*x[0] + x[1]*x[1]", element=V.ufl_element())
# f.ufl_element() is FiniteElement('Hermite', triangle, 3)
u = project(f, V)  # throws exception
_ = plot(u, cmap='bone')
print("f is a %s and u is a %s" % (f.ufl_element(), u.ufl_element()))

# Interpolation

As mentioned above, `interpolate()` cannot work, without extending the interface for `ufc::function` to include differentiation. This comes from the fact that the standard (local) nodal interpolant is given by

$$ I_K (f) = \sum_{i = 1}^{n_K} L_i (f) \phi_i $$

for all dofs $L_i$ over cell $K$. In particular we need to let the partial differentiation along each spatial coordinate (in the notation of the paper $L_{\alpha_i}, 1 \leq i \leq d$) act on the function $f$ and use this value as the coefficient for the corresponding shape function $\phi_{\alpha_i}$ in the representation in the global basis of $V_h$.

In [ ]:
#mesh = UnitSquareMesh(10,10)
#V = FunctionSpace(mesh, 'P', 3)
#f = phi1 - phi2
v = interpolate(phi3, V)
_ = plot(v, cmap='bone')

## A test

The first thing to check once `evaluate_dof()` has been implemented for Hermite elements is a constant:

In [ ]:
u = interpolate(Constant(2.0), V)
_ = plot(u, cmap='bone')

In [ ]:
_ = pl.plot([u(x, 0.98) for x in xx])

# Comparison to Lagrange elements

We project the same function onto P3:

In [ ]:
W = FunctionSpace(V.mesh(), 'P', 3)
f = Expression("x[0]*x[0] + x[1]*x[1]", degree=3)
u = project(f, W)
_ = plot(u, cmap='bone')

Projection of a constant looks much better than with Hermite elements:

In [ ]:
u = project(Constant(2.0), W)
_ = plot(u, cmap='bone')

# Extracting the nodal values

See section 5.2.6 of the tutorial.

In [ ]:
f = Expression("x[0]*x[0] + x[1]*x[1]", degree=3)
u = project(f, V)

vals = u.compute_vertex_values(V.mesh())
coordinates = V.mesh().coordinates()

import matplotlib.pyplot as pl
pl.scatter(coordinates.T[0], coordinates.T[1], c=vals, s=100, cmap='bone', linewidths=0)
_ = pl.axes().set_aspect('equal')

# Test construction of polynomial set

This is taken from the constructor of `FIAT.finite_element.FiniteElement()`.

**TODO:** I'd like to test whether the coefficients for the nodal basis functions agree with what I should be getting. 

In [ ]:
# build generalized Vandermonde matrix
from FIAT import polynomial_set, reference_element, hermite
K = reference_element.ReferenceElement(shape = reference_element.LINE,
                                       vertices = ((-1.0,),(1.0,)),
                                       topology = reference_element.UFCInterval().get_topology())
poly_set = polynomial_set.ONPolynomialSet(K, 3)
old_coeffs = poly_set.get_coeffs()
dual = hermite.CubicHermiteDualSet(K)
dualmat = dual.to_riesz(poly_set)

shp = dualmat.shape
if len(shp) > 2:
    num_cols = np.prod(shp[1:])

    A = np.reshape(dualmat, (dualmat.shape[0], num_cols))
    B = np.reshape(old_coeffs, (old_coeffs.shape[0], num_cols))
else:
    A = dualmat
    B = old_coeffs

V = np.dot(A, np.transpose(B))

Vinv = np.linalg.inv(V)

new_coeffs_flat = np.dot(np.transpose(Vinv), B)

new_shp = tuple([new_coeffs_flat.shape[0]] + list(shp[1:]))
new_coeffs = np.reshape(new_coeffs_flat, new_shp)

#poly_set = PolynomialSet(K, poly_set.get_degree(), poly_set.get_embedded_degree(),
#                         poly_set.get_expansion_set(), new_coeffs, poly_set.get_dmats())

In [ ]:
H = hermite.CubicHermite(K)
np.round(H.poly_set.tabulate_new(K.get_vertices()),2)

# Test transformation of basis functions

This is copied from `transform_hermite.py`, in a previous release of FIAT. It tests whether the nodal basis evaluated on a lattice over the reference element and over a transformed  element transforms well under an affine mapping.

Basically, what we do is we compute the "hermite" transformation in `evaluatebasis.py` by hand:

1. Tabulate (compute) the values of each basis function of the reference element over a lattice of points in the triangle, then multiply it by an adequate transformation matrix (mixing the values for the nodal basis functions related to derivatives).
2. Tabulate the values of each basis function defined over a different simplex.
3. Compare both results

In [ ]:
from FIAT import reference_element, hermite
from __future__ import print_function
import numpy as np

# Let's set up the reference triangle and another one
Khat = reference_element.UFCTriangle()
K = reference_element.ReferenceElement(shape = reference_element.TRIANGLE,
                                       vertices = ((-1.0,-1.0),(1.0,-1.0),(-1.0,1.0)),
                                       topology = Khat.get_topology())

# Construct the affine mapping between them
A, b = reference_element.make_affine_mapping(K.get_vertices(), Khat.get_vertices())

# build the Hermite element on the two triangles
Hhat = hermite.CubicHermite(Khat)
H = hermite.CubicHermite(K)

# get some points on each triangle
pts_hat = Khat.make_lattice(6)
pts = K.make_lattice(6)

# as a sanity check on the affine mapping, make sure pts map to pts_hat
for i, p in enumerate(pts):
    if not np.allclose(pts_hat[i], np.dot(A, p) + b):
        print("barf")

In [ ]:
# Tabulate the Hermite basis on each triangle
# Each column is the value of one nodal basis function evaluated at every point
Hhat_tabulated = Hhat.get_nodal_basis().tabulate_new(pts_hat)
H_tabulated = H.get_nodal_basis().tabulate_new(pts)

### Prepare global transformation matrix
M = np.zeros((10,10), dtype=np.double)
Ainv = np.linalg.inv(A)

# Point values are transformed as is
# Derivative values are "mixed" with the Jacobian.
M[0,0] = 1.0
M[3,3] = 1.0
M[6,6] = 1.0
M[9,9] = 1.0
M[1:3,1:3] = np.transpose(Ainv)
M[4:6,4:6] = np.transpose(Ainv)
M[7:9,7:9] = np.transpose(Ainv)

print(np.allclose(H_tabulated, np.dot(M.T, Hhat_tabulated)))

In [ ]:
Hhat_tabulated.shape

In [ ]:
basis = H.get_nodal_basis()
np.round(basis.tabulate_new([(-1,-1),(-1,0),(0,0)]))

## What are dmats ("expansion coefficients for basis function derivative") ?

In [ ]:
from FIAT import lagrange
Hhat = lagrange.Lagrange(Khat, 1)
basis = Hhat.get_nodal_basis()
dmats = basis.get_dmats()
print(len(dmats), dmats[0].shape, dmats[1].shape)
print(np.round(dmats[0], 1))
print(np.round(dmats[1], 1))